<img src="https://iteso.mx/documents/27014/202031/Logo-ITESO-MinimoH.png"
     align="right"
     width="300"/>

# Predicción TIIE 28 días utilizando Feedforward Neural Networks FNN

## *Modelos no lineales para pronósitico*  - Pedro Martinez

---

Una red neuronal de propagación hacia adelante FNN es una red neuronal en la que la información fluye en una sola dirección: las entradas se multiplican por pesos para obtener las salidas (entradas a salida). Puede utilizarse en un análisis de series de tiempo si los datos se preparan correctamente.

En este notebook aprenderemos a:
- Entender el perceptrón como base de las redes neuronales.
- Construir un dataset de ventanas (lags) a partir de una serie temporal.
- Entrenar una FNN para predecir valores futuros.
- Comparar los resultados con los valores reales de la TIIE (Banxico API).

La idea es que una red neuronal puede aprender **patrones no lineales** que los modelos tradicionales (ARIMA, SARIMA) no siempre capturan.

In [1]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go

In [2]:
# Serie: TIIE 28 días
serie_id = "SF43718"
token = "80996d887f4a37288c1c4f1a19b52f95306031c52d724f0c623eb1529a429f99"

url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos"
headers = {"Bmx-Token": token}
response = requests.get(url, headers=headers).json()

# Convertir a DataFrame
data = response["bmx"]["series"][0]["datos"]
df = pd.DataFrame(data)
df["fecha"] = pd.to_datetime(df["fecha"], format="%d/%m/%Y")
df["dato"] = pd.to_numeric(df["dato"], errors="coerce")
df = df.dropna().set_index("fecha")

df.tail()

,dato
fecha,
2025-09-29,18.3507
2025-09-30,18.3342
2025-10-01,18.3477
2025-10-02,18.4843
2025-10-03,18.3902


In [3]:
fecha_inicio = df.index.max() - pd.DateOffset(years=2)
df_2y = df[df.index >= fecha_inicio]

# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df_2y[["dato"]])

# Función para crear ventanas de tiempo
def create_dataset(series, window=5):
    X, y = [], []
    for i in range(len(series) - window):
        X.append(series[i:(i+window), 0])
        y.append(series[i+window, 0])
    return np.array(X), np.array(y)


# LSTM

def evaluar_ventanas_lstm(df_scaled, ventanas, n_test=15):
    """
    Prueba diferentes tamaños de ventana con un modelo LSTM.
    """
    resultados = {}
    print("Iniciando evaluación de ventanas para el modelo LSTM...")
    
    for w in ventanas:
        print(f"\n--- Probando ventana de tamaño: {w} ---")
        
        # Crear ventanas
        X, y = create_dataset(df_scaled, w)
        X_train, X_test = X[:-n_test], X[-n_test:]
        y_train, y_test = y[:-n_test], y[-n_test:]
        
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        
        # Crear y compilar el modelo LSTM
        model = Sequential()
        
        model.add(LSTM(50, activation="relu", input_shape=(w, 1))) 
        model.add(Dense(25, activation="relu"))
        model.add(Dense(1, activation="linear"))
        model.compile(optimizer="adam", loss="mse")
        
        # Se entrena el modelo
        model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0) 
        
        # Predecir y calcular métricas
        y_pred_scaled = model.predict(X_test)
        
        # Invertir la escala
        y_pred = scaler.inverse_transform(y_pred_scaled)
        y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

        rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
        mae = mean_absolute_error(y_test_real, y_pred)
        
        resultados[w] = {"RMSE": rmse, "MAE": mae}
        print(f"Resultados para ventana {w}: RMSE={rmse:.4f}, MAE={mae:.4f}")
    
    return resultados

# Evaluar diferentes tamaños de ventana
ventanas = [5, 10, 15, 30]
resultados_lstm = evaluar_ventanas_lstm(df_scaled, ventanas)

print("\n--- RESUMEN FINAL - MODELO LSTM ---")
print("Resultados por tamaño de ventana:")
for w, met in resultados_lstm.items():
    print(f"Ventana {w}: RMSE={met['RMSE']:.4f}, MAE={met['MAE']:.4f}")


Iniciando evaluación de ventanas para el modelo LSTM...

--- Probando ventana de tamaño: 5 ---


c:\Users\canal\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
Resultados para ventana 5: RMSE=0.0847, MAE=0.0697

--- Probando ventana de tamaño: 10 ---


c:\Users\canal\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Resultados para ventana 10: RMSE=0.0852, MAE=0.0741

--- Probando ventana de tamaño: 15 ---


c:\Users\canal\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
Resultados para ventana 15: RMSE=0.0763, MAE=0.0610

--- Probando ventana de tamaño: 30 ---


c:\Users\canal\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
Resultados para ventana 30: RMSE=0.0958, MAE=0.0798

--- RESUMEN FINAL - MODELO LSTM ---
Resultados por tamaño de ventana:
Ventana 5: RMSE=0.0847, MAE=0.0697
Ventana 10: RMSE=0.0852, MAE=0.0741
Ventana 15: RMSE=0.0763, MAE=0.0610
Ventana 30: RMSE=0.0958, MAE=0.0798


Podemos observar que con la comparación de MAEs, la mejor ventana es la de 30, por lo que usaremos ese valor.

In [ ]:
window_size = 30

# Se vuelve a crear el dataset con el tamaño nuevo
print(f"Creando el dataset final con window_size = {window_size}...")
X, y = create_dataset(df_scaled, window_size)

# De igual manera con el test
n_test = 15
X_train, X_test = X[:-n_test], X[-n_test:]
y_train, y_test = y[:-n_test], y[-n_test:]


fechas_test = df_2y.index[window_size:][-n_test:]


X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print("Datos listos para el entrenamiento final.")


model = Sequential()

model.add(LSTM(50, activation="relu", input_shape=(window_size, 1)))
model.add(Dense(25, activation="relu"))
model.add(Dense(1, activation="linear"))

model.compile(optimizer="adam", loss="mse")

history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=16, 
                    validation_data=(X_test, y_test), # Usar X_test, y_test para validar
                    verbose=1)

# Preedicciones con el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Se invierte la normalización
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

# Resultados finales
resultados = pd.DataFrame({'Real': y_test_real.flatten(), 'Predicho': y_pred.flatten()}, index=fechas_test)

Creando el dataset final con window_size = 30...
Datos listos para el entrenamiento final.
Epoch 1/50


c:\Users\canal\miniconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 0.1656 - val_loss: 7.4960e-04
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0078 - val_loss: 6.0809e-04
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0055 - val_loss: 4.1275e-04
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0047 - val_loss: 6.5062e-04
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0042 - val_loss: 9.9528e-04
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0038 - val_loss: 4.8495e-04
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0035 - val_loss: 7.4794e-04
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0039 - val_loss: 5.8928e-04
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0033 - val_loss: 0.0012
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0029 - val_loss: 7.0244e-04
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0028 - val_loss: 0.0011
Epoch 12/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/st

In [7]:
resultados = pd.DataFrame({
    'Real': y_test_real.flatten(),
    'Predicho': y_pred.flatten()
}, index=fechas_test)

resultados.head()

,Real,Predicho
fecha,,
2025-09-12,18.4757,18.606661
2025-09-15,18.3635,18.547716
2025-09-17,18.3257,18.459166
2025-09-18,18.3610,18.389153
2025-09-19,18.3892,18.369917


In [8]:
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_real = scaler.inverse_transform(y_test.reshape(-1, 1))

df_pred = pd.DataFrame({
    'Real': y_test_real.flatten(), 
    'Predicho': y_pred.flatten()
}, index=fechas_test)

# Empezamos a preparar datos para la gráfica 
X_reshaped = X.reshape((X.shape[0], X.shape[1], 1))
y_pred_all = model.predict(X_reshaped)

y_pred_all_rescaled = scaler.inverse_transform(y_pred_all)
y_all_rescaled = scaler.inverse_transform(y.reshape(-1, 1))

fechas_all = df_2y.index[window_size:]

# Crear el DataFrame con todas las predicciones
df_pred_all = pd.DataFrame({
    "Real": y_all_rescaled.flatten(),
    "Predicho": y_pred_all_rescaled.flatten()
}, index=fechas_all)

# Gráfica con Plotly
fig = go.Figure()

# Serie de 2 años real (la base gris)
fig.add_trace(go.Scatter(x=df_2y.index, y=df_2y["dato"],
                         mode="lines",
                         name="Serie Real (últimos 2 años)",
                         line=dict(color="lightgray")))

# Predicciones de todo el modelo (la línea naranja)
fig.add_trace(go.Scatter(x=df_pred_all.index, y=df_pred_all["Predicho"],
                         mode="lines",
                         name="Predicho (train + test)",
                         line=dict(color="orange")))

# Últimos 15 días - reales (línea azul con puntos)
fig.add_trace(go.Scatter(x=df_pred.index, y=df_pred["Real"],
                         mode="lines+markers",
                         name="Real (últimos 15 días)",
                         line=dict(color="#0047AB", width=2.5), # Azul cobalto
                         marker=dict(size=6)))

# Últimos 15 días - predichos (línea roja punteada)
fig.add_trace(go.Scatter(x=df_pred.index, y=df_pred["Predicho"],
                         mode="lines+markers",
                         name="Predicho (últimos 15 días)",
                         line=dict(color="#D32F2F", dash="dot", width=2.5), # Rojo fuerte
                         marker=dict(size=6)))

fig.update_layout(
    title="Predicción de la TIIE con Modelo LSTM",
    xaxis_title="Fecha",
    yaxis_title="TIIE 28 días (%)",
    legend=dict(x=0.01, y=0.99, bordercolor="black", borderwidth=1),
    template="plotly_white" # Un template limpio
)

fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
